In [1]:
# used to widen the cells 
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 2.5% !important; width:95%; }</style>"))


### Sandbox 

In [1]:
from algosdk.v2client import algod

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)


### PureStake API 

In [3]:
from algosdk.v2client import algod
from algosdk import mnemonic
from algosdk import transaction

algod_address = "https://testnet-algorand.api.purestake.io/ps2"
algod_token = "fFG6R4JTFq9Hh8RjZy130aMRdUhmOYp68hi9nm4L"
headers = {
   "X-API-Key": algod_token,
}

algod_client = algod.AlgodClient(algod_token, algod_address, headers);


In [4]:
def generate_algorand_keypair():
    private_key, address = account.generate_account()
    print("My address: {}".format(address))
    print("My passphrase: {}".format(mnemonic.from_private_key(private_key)))
    return {'sk': private_key, 'pk': address}

In [5]:
from algosdk import mnemonic, account

passphrase = "angry live wrist arrest nerve stadium round candy scrap area road defy grow panic turkey attack moon shadow they worth shoe forum rural ability average"

creator_account = {'sk': mnemonic.to_private_key(passphrase), 'pk': mnemonic.to_public_key(passphrase)}

account_info = algod_client.account_info(creator_account['pk'])
print("Account balance: {} microAlgos".format(account_info.get('amount')))


Account balance: 96162232 microAlgos


### Some funded accounts 

In [6]:
funded_mnemonics= ["rookie similar wire owner wash once fish mosquito glad coffee family venture adult funny melt gas inspire tuna buzz sell dignity pottery gold able bracket",
                  "narrow mimic suffer top suspect follow menu broccoli try snake meat erase napkin lucky client forget sense bread glad eight uniform bacon code about crack", 
                  "city truth device clog grocery sea safe slide glove borrow swap capable trash shaft vast start space distance calm wire hub crush dose ability army", 
                  "unknown arctic antenna country credit december ill practice lawsuit today athlete rescue exit swarm fitness strong minimum soldier wide coffee vacuum piece coil absorb unable", 
                  "gravity adult destroy demand margin coast culture base adjust east banana certain happy daughter bless fiscal fiscal eye forget inspire brain banner evil able bonus"]

In [7]:
funded_accounts = [{'sk': mnemonic.to_private_key(memo), 'pk': mnemonic.to_public_key(memo)} for memo in funded_mnemonics]

In [8]:
for ac in funded_accounts:
    print(algod_client.account_info(ac['pk']).get('amount'))

400396800
96036216
93447216
92735200
98581200


In [8]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema, OnComplete

## Creating asset

In [16]:
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

params.last

10546524

In [18]:
# creator_account creates a new asset 
# other_account as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction

txn = transaction.AssetConfigTxn(
    fee=params.fee,
    first=params.first, 
    last=params.last,
    gh=params.gh,
    flat_fee=params.flat_fee,
    sender=creator_account['pk'],
    total=1000, 
    default_frozen=False, 
    unit_name="QOIN", 
    asset_name="QVoteCoin", 
    manager=funded_accounts[0]['pk'],
    reserve=funded_accounts[0]['pk'],
    freeze=funded_accounts[0]['pk'],
    clawback=funded_accounts[0]['pk'],
    url='', 
    decimals=0,
)


# Sign with secret key of creator
stxn = txn.sign(creator_account['sk'])

# Send the transaction to the network and retrieve the txid.
txid = algod_client.send_transaction(stxn)
print(txid)

A5NLL236DH64ELIGYWTEYBEZOBYJ57BOMW2HI4DPPKJBABADNWSQ


In [21]:
tx_response = algod_client.pending_transaction_info(txid)
asset_id = tx_response['asset-index']
tx_response

{'asset-index': 13164495,
 'confirmed-round': 10545534,
 'pool-error': '',
 'txn': {'sig': '+YNfVyo1b5jaeaaZvrKXYt+QeZ4Brkr0hNDvHlm4NhX1ynHW9jSv8eEpO8DJfZ5HC8zwP+VguzHDx48snHo/Bw==',
  'txn': {'apar': {'an': 'QVoteCoin',
    'c': '2HJNF65MZIMT4YGTTDLGJK3LHS5B3IJUITWF3JJ3WC6GE72DLMJCYGZBUM',
    'f': '2HJNF65MZIMT4YGTTDLGJK3LHS5B3IJUITWF3JJ3WC6GE72DLMJCYGZBUM',
    'm': '2HJNF65MZIMT4YGTTDLGJK3LHS5B3IJUITWF3JJ3WC6GE72DLMJCYGZBUM',
    'r': '2HJNF65MZIMT4YGTTDLGJK3LHS5B3IJUITWF3JJ3WC6GE72DLMJCYGZBUM',
    't': 1000,
    'un': 'QOIN'},
   'fee': 1000,
   'fv': 10545524,
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 10546524,
   'snd': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM',
   'type': 'acfg'}}}

In [9]:
asset_id = 13164495

## Transferring the asset 

In [10]:
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True


### opt in to receive the asset 

In [21]:
# Use the AssetTransferTxn class to transfer assets and opt-in
receiver_account = funded_accounts[4]

"""txn = transaction.AssetTransferTxn(
    receiver_account['pk'],
    # params,
    first=params.first, 
    last=params.last,
    gh=params.gh,
    fee=params.fee,
    # flat_fee=params.flat_fee,
    receiver=receiver_account["pk"],
    amt=0,
    index=asset_id)"""

txn = transaction.AssetTransferTxn(
        sender=receiver_account['pk'],
        sp=params,
        receiver=receiver_account["pk"],
        amt=0,
        index=asset_id)

stxn = txn.sign(receiver_account['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)

IndexError: list index out of range

In [20]:
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

"""txn = transaction.AssetTransferTxn(
    sender=creator_account['pk'],     # this is the asset manager 
    # first=params.first, 
    # last=params.last,
    # gh=params.gh,
    # fee=params.fee,
    # flat_fee=params.flat_fee,
    params=params,
    receiver=receiver_account["pk"],
    amt=42,
    index=asset_id)"""

txn = transaction.AssetTransferTxn(
        sender=creator_account['pk'],
        sp=params,
        receiver=receiver_account["pk"],
        amt=93,
        index=asset_id)

stxn = txn.sign(creator_account['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)

3UQQV7QP6LAS4UFBQMVH77VX5KVRAMHXNVC6MPTCB7KUHGFTR2VA


## Reading token balance 

In [14]:
algod_client.account_info(creator_account['pk'])['assets'][-1]

{'amount': 885,
 'asset-id': 13164495,
 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM',
 'is-frozen': False}

In [15]:
algod_client.account_info(funded_accounts[1]['pk'])['assets']

[{'amount': 42,
  'asset-id': 13164495,
  'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM',
  'is-frozen': False}]

In [16]:
algod_client.account_info(funded_accounts[2]['pk'])['assets']

[{'amount': 73,
  'asset-id': 13164495,
  'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM',
  'is-frozen': False}]

In [25]:
for acc in funded_accounts:
    print(algod_client.account_info(acc['pk'])['assets'])

[]
[{'amount': 42, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]
[{'amount': 73, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]
[{'amount': 64, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]
[{'amount': 93, 'asset-id': 13164495, 'creator': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM', 'is-frozen': False}]


## Reading token balance in smart contract 

In [162]:
from pyteal import *    
    
def approval_program():     
    on_creation = Seq([    
        App.globalPut(Bytes("Creator"), Txn.sender()),    
        App.globalPut(Bytes("Name"), Txn.application_args[0]),
        App.globalPut(Bytes("asset_id"), Btoi(Txn.application_args[1])),  # should be Int 
        Return(Int(1))
    ])    
    
    on_closeout = Return(Int(1))
    
    asset_id = App.globalGet(Bytes("asset_id"))
    # parameter indicates index in accounts array (0 is sender)     
    asset_balance = AssetHolding.balance(Int(0), asset_id)    
        
    on_optin = Seq([ 
        asset_balance, 
        If(asset_balance.hasValue(),  # is sender has balance, store it, otherwise return 0     
           Seq([App.localPut(Int(0), Bytes("QVoteDecisionCredits"), asset_balance.value()), Return(Int(1))]),      
           Return(Int(0))
        ) 
    ])    
        
    # IF VOTING     
    # check balance of sender,     
    # add votes     
    # remove square of votes from asset balance for that address     
        
    program = Cond(    
            [Txn.application_id() == Int(0), on_creation],    
            [Txn.on_completion() == OnComplete.DeleteApplication, Return(Int(0))],   ## TODO does this mean no-one can delete this?         
            [Txn.on_completion() == OnComplete.UpdateApplication, Return(Int(0))],    
            [Txn.on_completion() == OnComplete.OptIn, on_optin]    
            # TODO add voting     
    )    
    return program    
    
    
# TODO add timeframes for opt in and voting     
# TODO does it matter where the balance of tokens is stored? global vs local     
    
def clear_state_program():     
    program = Return(Int(1))
    return program    
    


In [163]:
approval_filename = 'approval.teal'
clear_state_filename = 'clear_state.teal'

with open(approval_filename, 'w') as f:
    compiled = compileTeal(approval_program(), Mode.Application)
    f.write(compiled)

with open(clear_state_filename, 'w') as f:
    compiled = compileTeal(clear_state_program(), Mode.Application)
    f.write(compiled)

In [164]:
stdout, stderr = execute(["goal", "clerk", "compile", "-o", approval_filename+'.tok', approval_filename])
if stderr != "":
    print(stderr)
    raise
elif len(stdout) < 59:
    print("error in compile teal")
    raise
    
stdout, stderr = execute(["goal", "clerk", "compile", "-o", clear_state_filename+'.tok', clear_state_filename])
if stderr != "":
    print(stderr)
    raise
elif len(stdout) < 59:
    print("error in compile teal")
    raise

In [165]:
with open(approval_filename+'.tok', 'rb') as f:
    approval_bytes = f.read()
    
with open(clear_state_filename+'.tok', 'rb') as f: 
    clear_state_bytes = f.read() 

In [166]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema, OnComplete

In [167]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

decision_name = 'to_be_or_not_to_be'

local_schema = StateSchema(num_uints=1, num_byte_slices=1)    
global_schema = StateSchema(num_uints=3, num_byte_slices=3)

on_complete = OnComplete(0)
app_create_txn = transaction.ApplicationCreateTxn(
    funded_accounts[2]['pk'], 
    params, 
    on_complete, 
    approval_bytes, 
    clear_state_bytes, 
    global_schema, 
    local_schema,
    app_args = [decision_name.encode('utf-8'), asset_id.to_bytes(3, 'big')]
)

In [168]:
app_create_txn_signed = app_create_txn.sign(funded_accounts[2]['sk'])
txid = algod_client.send_transaction(app_create_txn_signed)
txid

'KRZJNNVDUI7RNWRNO6YXJLK7VLFB44KY6QXSSD4G6ZXS7SQTLKKQ'

In [170]:
app_id = algod_client.pending_transaction_info(txid)['application-index']
app_id

13166035

## Opting in 

In [171]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000

# funded_accounts[1] has some tokens 
optin_tx = transaction.ApplicationOptInTxn(funded_accounts[1]['pk'], params, app_id)
optin_txid = algod_client.send_transaction(optin_tx.sign(funded_accounts[1]['sk']))

In [173]:
optin_tx_response = algod_client.pending_transaction_info(optin_txid)
optin_tx_response

{'confirmed-round': 10550054,
 'local-state-delta': [{'address': 'WSBV354QUX5TBQ7KNBRVCRMAFXJOJLKFIX46DIO2CP5BB7VQPHTCOKNGHQ',
   'delta': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
     'value': {'action': 2, 'uint': 42}}]}],
 'pool-error': '',
 'txn': {'sig': 'vikaoHYwPoE4upmfq1bJ/+VB1vPRKTPp2dA6ofWBQEa5CFdb/xebsIOB7zJHrY5F245sZp3MUCEZqX1loOl4Bw==',
  'txn': {'apan': 1,
   'apid': 13166035,
   'fee': 217000,
   'fv': 10550052,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 10551052,
   'snd': 'WSBV354QUX5TBQ7KNBRVCRMAFXJOJLKFIX46DIO2CP5BB7VQPHTCOKNGHQ',
   'type': 'appl'}}}

## Checking local storage 

In [180]:
# did anything actually change in the local storage? 
local_storage = algod_client.account_info(funded_accounts[1]['pk'])['apps-local-state']
local_storage

[{'id': 13166035,
  'key-value': [{'key': 'UVZvdGVEZWNpc2lvbkNyZWRpdHM=',
    'value': {'bytes': '', 'type': 2, 'uint': 42}}],
  'schema': {'num-byte-slice': 1, 'num-uint': 1}}]